# Exemplo dados do ***Big Query***

## Carregar pacotes

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import auth
from google.cloud import bigquery

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

## Autenticar projeto

In [ ]:
auth.authenticate_user()

In [ ]:
project_id = 'aula01in'

In [ ]:
client = bigquery.Client(project=project_id)

## Dados

### Partidas

buscar dados do *Big Query*

In [ ]:
dados_partidas_bq = client.query('''
  select *
  from `cartola_partidas.partidas_*` ''')

Transformar em pandas dataframe

In [ ]:
dados_partidas = dados_partidas_bq.to_dataframe()

In [ ]:
dados_partidas.ano

0       2017
1       2017
2       2017
3       2017
4       2017
        ... 
2275    2016
2276    2016
2277    2016
2278    2016
2279    2016
Name: ano, Length: 2280, dtype: int64

### Classificação

In [ ]:
dados_classificacao_bq = client.query('''
  select *
  from `cartola_classificacao.classificacao_*` ''')

In [ ]:
dados_classificacao = dados_classificacao_bq.to_dataframe()

In [ ]:
dados_classificacao.ano

0       2017
1       2017
2       2017
3       2017
4       2017
        ... 
4555    2016
4556    2016
4557    2016
4558    2016
4559    2016
Name: ano, Length: 4560, dtype: int64

### Jogadores

In [ ]:
dados_jogadores_bq = client.query('''
  select *
  from `cartola_scouts.scouts_*` ''')

In [ ]:
dados_jogadores = dados_jogadores_bq.to_dataframe()

In [ ]:
dados_jogadores.head()

,atleta_id,apelido,rodada_id,ano,clube_id,clube,posicao_id,posicao,jogos_num,pontos_num,media_num,preco_num_anterior,preco_num,variacao_num,FS,PE,A,FT,FD,FF,G,I,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS,status_id,status
0,42499,Bruno,14,2018,265,Bahia,2,lat,0,2.5,0.30,2.69,3.39,0.70,2,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,6,Nulo
1,42499,Bruno,22,2018,265,Bahia,2,lat,0,2.0,1.22,3.44,3.37,-0.07,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,6,Nulo
2,94980,Geovane Itinga,11,2018,265,Bahia,5,ata,0,0.5,0.50,2.00,1.60,-0.40,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,Nulo
3,61188,Gilberto,14,2018,265,Bahia,5,ata,0,12.7,10.30,9.02,12.28,3.26,4,1,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,6,Nulo
4,61188,Gilberto,36,2018,265,Bahia,5,ata,0,0.0,5.31,12.20,12.05,-0.15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,Nulo


## Selecionar dados para criação da *view*

In [ ]:
colunas_partidas = ['rodada_id', 'ano', 'clube_casa_id', 'clube_visitante_id', 'partidas_local']
partidas_view = dados_partidas[colunas_partidas]
partidas_view.head()

,rodada_id,ano,clube_casa_id,clube_visitante_id,partidas_local
0,11,2017,314,303,Ressacada
1,15,2017,314,264,Ressacada
2,16,2017,314,283,Ressacada
3,19,2017,314,277,Ressacada
4,22,2017,314,315,Ressacada


In [ ]:
colunas_jogadores = ['atleta_id', 'apelido', 'rodada_id', 'ano', 'clube_id', 'clube',
                     'posicao', 'pontos_num', 'preco_num']
players_view = dados_jogadores[colunas_jogadores]
players_view.head()

,atleta_id,apelido,rodada_id,ano,clube_id,clube,posicao,pontos_num,preco_num
0,42499,Bruno,14,2018,265,Bahia,lat,2.5,3.39
1,42499,Bruno,22,2018,265,Bahia,lat,2.0,3.37
2,94980,Geovane Itinga,11,2018,265,Bahia,ata,0.5,1.60
3,61188,Gilberto,14,2018,265,Bahia,ata,12.7,12.28
4,61188,Gilberto,36,2018,265,Bahia,ata,0.0,12.05


In [ ]:
colunas_classificacao = ['clube_id', 'rodada_id', 'ano', 'clube_rank']
classif_view = dados_classificacao[colunas_classificacao]
classif_view.head()

,clube_id,rodada_id,ano,clube_rank
0,262,12,2017,2
1,262,10,2017,3
2,262,11,2017,3
3,262,2,2017,3
4,262,13,2017,4


In [ ]:
players_view = pd.merge(players_view, classif_view,  how = 'left', on=['clube_id', 'rodada_id', 'ano'])
players_view.head()

,atleta_id,apelido,rodada_id,ano,clube_id,clube,posicao,pontos_num,preco_num,clube_rank
0,42499,Bruno,14,2018,265,Bahia,lat,2.5,3.39,14
1,42499,Bruno,22,2018,265,Bahia,lat,2.0,3.37,14
2,94980,Geovane Itinga,11,2018,265,Bahia,ata,0.5,1.60,18
3,61188,Gilberto,14,2018,265,Bahia,ata,12.7,12.28,14
4,61188,Gilberto,36,2018,265,Bahia,ata,0.0,12.05,11


In [ ]:
players_casa = pd.merge(players_view, partidas_view,  how = 'right', 
                        left_on=['clube_id', 'rodada_id', 'ano'], 
                        right_on=['clube_casa_id', 'rodada_id', 'ano'])
players_casa.head()

,atleta_id,apelido,rodada_id,ano,clube_id,clube,posicao,pontos_num,preco_num,clube_rank,clube_casa_id,clube_visitante_id,partidas_local
0,42499.0,Bruno,14,2018,265.0,Bahia,lat,2.5,3.39,14.0,265,287,Fonte Nova
1,61188.0,Gilberto,14,2018,265.0,Bahia,ata,12.7,12.28,14.0,265,287,Fonte Nova
2,82455.0,Zé Rafael,14,2018,265.0,Bahia,mei,14.3,20.72,14.0,265,287,Fonte Nova
3,74271.0,Edigar Junio,14,2018,265.0,Bahia,ata,-0.3,7.62,14.0,265,287,Fonte Nova
4,71719.0,Elton,14,2018,265.0,Bahia,mei,6.0,9.65,14.0,265,287,Fonte Nova


In [ ]:
players_fora = pd.merge(players_view, partidas_view,  how = 'right', 
                        left_on=['clube_id', 'rodada_id', 'ano'], 
                        right_on=['clube_visitante_id', 'rodada_id', 'ano'])
players_fora.head()

,atleta_id,apelido,rodada_id,ano,clube_id,clube,posicao,pontos_num,preco_num,clube_rank,clube_casa_id,clube_visitante_id,partidas_local
0,42499.0,Bruno,22,2018,265.0,Bahia,lat,2.0,3.37,14.0,293,265,Arena da Baixada
1,81770.0,Clayton,22,2018,265.0,Bahia,ata,-0.3,4.42,14.0,293,265,Arena da Baixada
2,102609.0,Fernando Castro,22,2018,265.0,Bahia,gol,-1.3,0.76,14.0,293,265,Arena da Baixada
3,78584.0,Douglas Friedrich,22,2018,265.0,Bahia,gol,10.5,4.58,14.0,293,265,Arena da Baixada
4,94980.0,Geovane Itinga,22,2018,265.0,Bahia,ata,0.0,1.60,14.0,293,265,Arena da Baixada


Ajustes casa e fora

In [ ]:
players_view = players_casa.append(players_fora)
players_view.head()

,atleta_id,apelido,rodada_id,ano,clube_id,clube,posicao,pontos_num,preco_num,clube_rank,clube_casa_id,clube_visitante_id,partidas_local
0,42499.0,Bruno,14,2018,265.0,Bahia,lat,2.5,3.39,14.0,265,287,Fonte Nova
1,61188.0,Gilberto,14,2018,265.0,Bahia,ata,12.7,12.28,14.0,265,287,Fonte Nova
2,82455.0,Zé Rafael,14,2018,265.0,Bahia,mei,14.3,20.72,14.0,265,287,Fonte Nova
3,74271.0,Edigar Junio,14,2018,265.0,Bahia,ata,-0.3,7.62,14.0,265,287,Fonte Nova
4,71719.0,Elton,14,2018,265.0,Bahia,mei,6.0,9.65,14.0,265,287,Fonte Nova


In [ ]:
players_view['mandante'] = np.where(players_view['clube_id'] == players_view['clube_casa_id'], 'S', 'N')
players_view.head()

,atleta_id,apelido,rodada_id,ano,clube_id,clube,posicao,pontos_num,preco_num,clube_rank,clube_casa_id,clube_visitante_id,partidas_local,mandante
0,42499.0,Bruno,14,2018,265.0,Bahia,lat,2.5,3.39,14.0,265,287,Fonte Nova,S
1,61188.0,Gilberto,14,2018,265.0,Bahia,ata,12.7,12.28,14.0,265,287,Fonte Nova,S
2,82455.0,Zé Rafael,14,2018,265.0,Bahia,mei,14.3,20.72,14.0,265,287,Fonte Nova,S
3,74271.0,Edigar Junio,14,2018,265.0,Bahia,ata,-0.3,7.62,14.0,265,287,Fonte Nova,S
4,71719.0,Elton,14,2018,265.0,Bahia,mei,6.0,9.65,14.0,265,287,Fonte Nova,S


In [ ]:
players_view.sort_values(['atleta_id', 'rodada_id', 'ano'], inplace=True)
players_view.head()

,atleta_id,apelido,rodada_id,ano,clube_id,clube,posicao,pontos_num,preco_num,clube_rank,clube_casa_id,clube_visitante_id,partidas_local,mandante
38268,36443.0,Dida,1,2014,285.0,Internacional,gol,5.0,10.60,6.0,285,287,Estádio Beira-Rio,S
22357,36443.0,Dida,1,2015,285.0,Internacional,gol,0.0,5.00,20.0,293,285,Arena da Baixada,N
37893,36443.0,Dida,2,2014,285.0,Internacional,gol,-3.0,8.27,6.0,263,285,Estádio Maracanã,N
23089,36443.0,Dida,2,2015,285.0,Internacional,gol,0.0,5.00,12.0,285,314,Beira-Rio,S
38207,36443.0,Dida,3,2014,285.0,Internacional,gol,-2.6,6.81,3.0,285,292,Beira-Rio,S


In [ ]:
colunas_view = ['apelido', 'posicao', 'rodada_id', 'ano', 'clube', 'clube_rank', 
                'mandante', 'partidas_local', 'pontos_num', 'preco_num']
players_view = players_view[colunas_view].reset_index(drop=True)

In [ ]:
players_view['apelido'] = players_view['apelido'].astype(str)
players_view.head()

,apelido,posicao,rodada_id,ano,clube,clube_rank,mandante,partidas_local,pontos_num,preco_num
0,Dida,gol,1,2014,Internacional,6.0,S,Estádio Beira-Rio,5.0,10.60
1,Dida,gol,1,2015,Internacional,20.0,N,Arena da Baixada,0.0,5.00
2,Dida,gol,2,2014,Internacional,6.0,N,Estádio Maracanã,-3.0,8.27
3,Dida,gol,2,2015,Internacional,12.0,S,Beira-Rio,0.0,5.00
4,Dida,gol,3,2014,Internacional,3.0,S,Beira-Rio,-2.6,6.81


##Gravar a View

In [ ]:
dataset_ref = client.dataset('TEMP')

In [ ]:
table_ref = dataset_ref.table("players_view")

Tipo 1 - somente atualização

In [ ]:
#job = client.load_table_from_dataframe(players_view, table_ref, 
                                       #job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
#job.result() 

Tipo 2 - Atualiza e mantém adicionando as novas linhas

In [ ]:
job = client.load_table_from_dataframe(players_view, table_ref)
job.result() 

/usr/local/lib/python3.6/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'apelido'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))
